In [18]:
import numpy as np
import cv2
import os
import random
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, \
    InputLayer,Conv2D,MaxPool2D,UpSampling2D,Conv2DTranspose,Cropping2D,add,average
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [19]:
img_shape=(480,320,3)
# The encoder 1
encoder1_1 = Input(img_shape,name='encoder1_input')
encoder1_2 = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',name='E1Layer1')(encoder1_1)
encoder1_3 = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',name='E1Layer2')(encoder1_2)
encoder1_4 = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',name='E1Layer3')(encoder1_3)
# encoder1_5 = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',name='E1Layer4')(encoder1_4)

encoder1 = Model(encoder1_1,encoder1_4)

# The decoder 1
decoder1_1 = Input(encoder1.get_layer('E1Layer3').output_shape[1:],name='decoder1_input')
# decoder1_2 = Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='same',name='D1Layer1')(decoder1_1)
decoder1_3 = Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='same',name='D1Layer2')(decoder1_1)
decoder1_4 = Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='same',name='D1Layer3')(decoder1_3)
decoder1_5 = Conv2DTranspose(filters=3,kernel_size=3,strides=2,padding='same',name='D1Layer4')(decoder1_4)

decoder1 = Model(decoder1_1,decoder1_5)

inp = Input(img_shape)
code = encoder1(inp)
reconstruction = decoder1(code)
autoencoder1 = Model(inp,reconstruction)

##################################
####### Second Autoencoder #######
##################################

# The encoder 2
encoder2_1 = Input(img_shape,name='encoder2_input')
encoder2_2 = Conv2D(filters=256,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='E2Layer1')(encoder2_1)
encoder2_3 = Conv2D(filters=128,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='E2Layer2')(encoder2_2)
encoder2_4 = Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='E2Layer3')(encoder2_3)
# encoder2_5 = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='E2Layer4')(encoder2_4)

# The decoder 2
decoder2_2 = Conv2DTranspose(filters=128,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='D2Layer1')(encoder2_4)
add2_1 = average([decoder2_2,encoder2_3]) # Residual Connection 
# decoder2_3 = Conv2DTranspose(filters=128,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='D2Layer2')(add2_1)
decoder2_4 = Conv2DTranspose(filters=256,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='D2Layer3')(add2_1)
add2_2 = average([decoder2_4,encoder2_2]) # Residual Connection 
decoder2_5 = Conv2DTranspose(filters=3,kernel_size=3,strides=2,padding='same',activation='relu',activity_regularizer=l1(10e-10),name='D2Layer4')(add2_2)

autoencoder2 = Model(encoder2_1,decoder2_5)

In [20]:
autoencoder2.summary()

Model: "functional_52"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder2_input (InputLayer)     [(None, 480, 320, 3) 0                                            
__________________________________________________________________________________________________
E2Layer1 (Conv2D)               (None, 240, 160, 256 7168        encoder2_input[0][0]             
__________________________________________________________________________________________________
E2Layer2 (Conv2D)               (None, 120, 80, 128) 295040      E2Layer1[0][0]                   
__________________________________________________________________________________________________
E2Layer3 (Conv2D)               (None, 60, 40, 64)   73792       E2Layer2[0][0]                   
______________________________________________________________________________________

In [10]:
(480*320*3)/(60*40*32)

6.0

In [ ]:
# PATHS
DATASET = r'C:\Users\saad\Desktop\Autoencoders\Videos\Dataset'  # where there is dataset
DATASET_NAME = ['bunny_video.mp4','Rabbit_video.mp4','bicycle_sequence.mp4'] # Name of the video
# where you want to save the frames
FRAMES = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results'
FRAME_NAME = ['bunny','rabbit','bicycle']
FRAME_TRAIN =['bunny_train','rabbit_train','bicycle_train']
FRAME_TEST = ['bunny_test','rabbit_test','bicycle_test']
names = [0 , 0 , 0]

In [ ]:
#Creating the video frames for each video
for idx,f in enumerate(FRAME_NAME):
    names[idx],fps,width,height = read_video(os.path.join(FRAMES,f),os.path.join(DATASET,DATASET_NAME[idx]))
    print(f'{f} completed')

#Randomly selecting certain number of images

In [ ]:
# 0 is for bunny
# 1 is for rabbit
# 2 is for bicycle

from shutil import copy

for i in tqdm(range(len(names)),desc='categorization'):
    
    train,test = splitter(np.array(names[i]),0.2)
    
    for j in tqdm(range(len(train)),desc='Training Data'):
        copy(os.path.join(FRAMES,FRAME_NAME[i],train[j]),os.path.join(FRAMES,FRAME_TRAIN[i]))
        
    for k in tqdm(range(len(test)),desc='Testing Data'):
        copy(os.path.join(FRAMES,FRAME_NAME[i],test[k]),os.path.join(FRAMES,FRAME_TEST[i]))

In [ ]:
train =[]
test = []

for j in tqdm(range(len(train)),desc='Training Data'):
    copy(os.path.join(FRAMES,FRAME_NAME[i],train[j]),os.path.join(FRAMES,FRAME_TRAIN[i]))
        
    for k in tqdm(range(len(test)),desc='Testing Data'):
        copy(os.path.join(FRAMES,FRAME_NAME[i],test[k]),os.path.join(FRAMES,FRAME_TEST[i]))


In [ ]:
os.path.join(FRAMES,FRAME_NAME[0],FRAME_TRAIN[0])

In [ ]:
images = []
for n in tqdm(FRAME_TRAIN):
    foo = os.path.join(FRAMES,n)
    im = np.array(read_imgs(foo,os.listdir(foo),2,640,480))
    images.append(im)

In [ ]:
images_test = []
for n in tqdm(FRAME_TEST):
    foo = os.path.join(FRAMES,n)
    im = np.array(read_imgs(foo,os.listdir(foo),2,640,480))
    images_test.append(im)

In [ ]:
images1 = np.array(images)

In [ ]:
images1[0].shape,images1[1].shape,images1[2].shape

In [ ]:
images2= images2.flatten()

In [ ]:
images2.shape

In [ ]:
flat_list = [item for sublist in images for item in sublist]

In [ ]:
f = np.array(flat_list)
f.shape

In [ ]:
image_test1 = [item for sublist in images_test for item in sublist]
image_test2 = np.array(image_test1)
image_test2.shape

In [ ]:
model1,model2 = build_autoencoder(f.shape[1:],512,'Hi-Res',None)

In [ ]:
model1.compile(loss='mse',optimizer='adam',metrics=['mae'])
model2.compile(loss='mse',optimizer='adam',metrics=['mae'])

In [ ]:
WEIGHTS = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Weights'
model1 = model_fit(model1,'model1_combined',WEIGHTS,f,f,200,5)

In [ ]:
train_intermediate = data_prep(f,model1,None)

In [ ]:
model2 = model_fit(model2,'model2_combined',WEIGHTS,train_intermediate,f,200,5)

In [ ]:
predictions2 = data_prep(image_test2,model1,model2)
# predictions = predictions * 255.

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt
ids = 299

plt.subplot(131)
plt.imshow(image_test2[ids,:,:,:])
plt.title('Original')

plt.subplot(132)
x = model1.predict(np.expand_dims(image_test2[ids],axis=0))
x = x.reshape(480,320,3)
plt.imshow(x.astype(int))
plt.title('Model1 Output')

plt.subplot(133)
y = predictions2[ids]
plt.imshow(y.astype(int))
plt.title('Final Output')

In [ ]:
image_test2[:1,:,:,:].shape

In [ ]:
g = model1.predict(np.expand_dims(f[0],axis=0))
g=g.reshape(480,320,3)
plt.imshow(g.astype(int))

In [ ]:
plt.imshow(f[0])

In [ ]:
model1.load_weights(os.path.join(WEIGHTS,'model1_combined.hdf5'))
model2.load_weights(os.path.join(WEIGHTS,'model2_combined.hdf5'))

In [ ]:
os.chdir(os.path.join(FRAMES,FRAME_NAME[1]))
os.getcwd()

In [ ]:
images = np.array(read_imgs(os.getcwd(),os.listdir(os.getcwd()),2,640,480))

In [ ]:
images.shape

In [ ]:
predictions_new1 = data_prep(images,model1,model2)

In [ ]:
video_creater('Recon_rabbit_new1.avi',os.getcwd(),25.,640,480,predictions_new1)

In [ ]:
path = r'C:\Users\saad\Desktop\Autoencoders\Videos\Dataset'
vid_name = 'Bicycle_Sequence.mp4'

In [ ]:
names,fps,width,height=read_video(os.getcwd(),os.path.join(path,vid_name))

In [ ]:
print(f'The fps is {fps}, The width of the frame is {width} and the height of the frame is {height}')

In [ ]:
names

In [ ]:
images = np.array(read_imgs(os.getcwd(),names,2,640,480))
images = images.astype('float32')/255.
print(f'The shape of the images is {images.shape}')

In [ ]:
img_shape = images.shape[1:]
print(img_shape)

In [ ]:
model1,model2 = build_autoencoder(img_shape,512,'Hi-Res',None)

In [ ]:
weights = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Weights'
weights = weights.replace('\\','/')


In [ ]:
model1.load_weights(os.path.join(weights,'model1.hdf5'))
model2.load_weights(os.path.join(weights,'model2.hdf5'))

In [ ]:
img_new = model1.predict(images)

In [ ]:
img_new.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_new[75,:,:,:])

In [ ]:
from tqdm import tqdm
import numpy as np
preds=[]
for i in tqdm(range(len(img_new))):
    preds.append(model2.predict(np.expand_dims(img_new[i],axis=0)))

In [ ]:
pr = np.array(preds)
pr.shape

In [ ]:
pr= pr.reshape(224,480,320,3)

In [ ]:
plt.imshow(pr[75,:,:,:])

In [ ]:
plt.imshow(images[75,:,:,:])

In [ ]:
im = 3
plt.figure(figsize=(14,10))

plt.subplot(131)
plt.imshow(images[im,:,:,:])
plt.title('Original')

plt.subplot(132)
plt.imshow(img_new[im,:,:,:])
plt.title('Intermediate')

plt.subplot(133)
plt.imshow(pr[im,:,:,:])
plt.title('Reconstructed')

plt.show()

In [ ]:
predictions = pr * 255.

In [ ]:
VIDEO = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Video'
video_creater('Recon_bicycle1.avi',VIDEO,25.,640,480,predictions) 

In [ ]:
 original_video('Original_bicycle.avi',VIDEO,os.getcwd(),25.,names)

In [ ]:
os.getcwd()

In [ ]:
lst=['a','b','c']

for i,j in enumerate(lst):
    print('i:',i)
    print('j:',j)